In [4]:
import numpy as np
import pandas as pd
import sys, re, os

%config Completer.use_jedi = False

In [5]:
df = pd.read_excel("initial_docs/LT_Recipient_MBPAKI_afterExclusion.xlsx", skiprows=7)
df.head()

,Case_ID,frac1,frac2,frac3,OPdate,ABLEID,Age,Room,Sex,Ht,...,IHM,Final_IHM,HCC_SUD,Final_HCC,Recur_death,HCC_date,HCCrecur,HBV재발,HBV재발_date,SalvageLT
0,0,0.0,0.0,NaN,2011-07-01,1195890799,49,D02,1,155.3,...,NaN,1.0,692.0,2.0,2013-05-23,2013-05-23,2.0,NaN,NaT,1.0
1,1,0.0,0.0,NaN,2011-07-05,1196102763,48,D02,2,166.6,...,NaN,1.0,771.0,2.0,2013-08-14,2013-08-14,2.0,2.0,2013-11-27,2.0
2,2,0.0,0.0,NaN,2011-07-05,1197701254,55,D06,2,168.3,...,NaN,1.0,119.0,2.0,2011-11-01,2011-11-01,2.0,NaN,NaT,1.0
3,3,0.0,0.0,NaN,2011-07-06,1197157572,42,D06,2,173.3,...,NaN,1.0,3159.0,1.0,NaT,NaT,NaN,NaN,NaT,1.0
4,4,0.0,0.0,NaN,2011-07-06,1183766718,57,D02,2,171.2,...,NaN,1.0,3159.0,1.0,NaT,NaT,NaN,NaN,NaT,1.0


    Criteria
    
        - 가장일반적인 간이식 수술종류만을 선택
            BX-CE 열 : OLT-1, ABOi-1, Dual-1, Re_TPL-1, EM_Elec-2, Donor_exchange-1, C_split-1, PED-1
        - 수술 대량출혈환자를 제외
            DE 열 MassTF-1만 선택
        - CaseID 열부터 DBP_mean7열까지 선택

In [6]:
temp = df[(df['MassTF']==1) & (df['OLT']==1) & (df['ABOI']==1) &\
           (df['Dual']==1) & (df['Re_TPL']==1) & (df['EM_Elec']==2) &\
           (df['Donor_exchange']==1) & (df['C_split']==1) & \
           (df['PED']==1)]
data = temp.loc[:, 'Case_ID':'DBP_mean7']
display(data.shape)

(1250, 164)

#### 결측값 
 - 25% 이상되는 변수들은 무엇인가?

In [7]:
percent_missing = data.isnull().sum() / len(data)
missing_value_data = pd.DataFrame({'column_name': data.columns,
                                 'ratio_missing': percent_missing})
missing_value_data[missing_value_data['ratio_missing']>0.25]

,column_name,ratio_missing
frac3,frac3,0.7456
TPE,TPE,0.8952
FMS,FMS,0.9864
Cell_saver,Cell_saver,0.8784
Levo_mcg,Levo_mcg,0.9864
Cr_postreper,Cr_postreper,0.5168
Tnl_preah,Tnl_preah,0.9856
TnI_pod2,TnI_pod2,0.3992
TnI_pod3,TnI_pod3,0.6504
TnI_pod4,TnI_pod4,0.7736


1. Correlation Coefficient가 0.85이상임 (왼족변수 삭제, 오른쪽 변수 남김)

BSA ~ Wt

TIT ~ CIT

CTP ~ MELD

CTPGr ~ MELD

MELD_Na ~ MELD

INR_MELD ~ MELD

Alb_CTP ~ Alb

FFP ~ RBC

Pre_Vasopressin ~ Pre_Epinephrine

AP_PC ~ AP

op_Norepi ~ Op_Ino


2. 다음의 변수는 값의 종류가 한가지임 

CKD, HRS, preRRT, Renal_replace_type, Pre_Dobutamine, Pre_Dopamine, Pre_Terlipressin, Pre_Isoket, IntraCRRT, op_Dopaf

3. post LT operation variables

TnI_pod2, TnI_pod3, TnI_pod4, TnI_pod5, TnI_pod6, TnI_pod7, post_RBC, post_FFP, Cryo_post, AP_post, PC_post

In [8]:
data.drop(columns = ['frac1', 'frac2', 'frac3', 'ABLEID', 'MassTF', 'OLT', 
                     'ABOI', 'Dual', 'Re_TPL',  'EM_Elec', 'Donor_exchange',
                     'C_split', 'PED', 'AKI', 'AKIgr', 'EAD_OLT', 'EAD', 
                     'TPE', 'FMS', 'Cell_saver', 'Cr_postreper', 'Tnl_preah',
                     'Case_ID', '마취시작시간', '마취종료시간', 'OPdate', 'MELD_Na',
                     'CTP', 'CTPGr', 'BSA', 'TB_MELD', 'INR_MELD', 'Alb_CTP', 
                     'TIT', 'AP_PC', 'op_Norepi', 'TnI_pod2', 'TnI_pod3',
                     'TnI_pod4', 'TnI_pod5', 'TnI_pod6', 'TnI_pod7', 'post_RBC', 
                     'post_FFP', 'Cryo_post', 'AP_post', 'PC_post'], inplace=True)
display(data.shape)

(1250, 117)

In [9]:
# 사건의 단위들은 모두 hour변경
data['Reperfusion'] = pd.to_datetime(data['Reperfusion'])
data['opBgn2Reperf_hr'] = (data['Reperfusion'] - data['R_Proc시작_able'])/ np.timedelta64(1, 'h')
data['Reperf2opTer_hr'] = (data['R_Proc종료_able'] - data['Reperfusion'])/ np.timedelta64(1, 'h')
data['anes_dur_hr'] = data['ANES_time'] / 60.
data['op_bgn_hr'] = data['수술시작일시'].dt.hour
data['op_ter_hr'] = data['수술종료시간'].dt.hour
data['op_mon'] = data['수술시작일시'].dt.month
data['op_yr'] = data['수술시작일시'].dt.year
data['CIT'] = data['CIT']/60
data['WIT'] = data['WIT']/60


In [10]:
data.drop(columns=['R_Proc시작_able', 'R_Proc종료_able', 'ICE들어간시간', 'ICE나온시간', 'Reperfusion', 
                  'ANES_time', 'OP_time', '수술시작일시', '수술종료시간', 'CKD', 'HRS', 'preRRT', 
                  'Renal_replace_type', 'Pre_Dobutamine', 'Pre_Dopamine', 'Pre_Terlipressin', 
                  'Pre_Isoket', 'IntraCRRT', 'op_Dopa', 'Pre_Vasopressin'], inplace=True)

In [11]:
data.columns

Index(['Age', 'Room', 'Sex', 'Ht', 'Wt', 'BMI', 'Cr_MELD', 'Na_MELD', 'MELD',
       'DM_Tx',
       ...
       'HR_mean7', 'SBP_mean7', 'DBP_mean7', 'opBgn2Reperf_hr',
       'Reperf2opTer_hr', 'anes_dur_hr', 'op_bgn_hr', 'op_ter_hr', 'op_mon',
       'op_yr'],
      dtype='object', length=104)

#### Room 숫자 코딩

In [12]:
data['Room'].value_counts()

D02    680
D06    567
D01      3
Name: Room, dtype: int64

In [13]:
data['Room'] = data['Room'].apply(lambda x: re.search(r'D([0-9]+)', x).group(1))

In [14]:
data['Room'] = data['Room'].astype(int)

#### GFR_MDRD 숫자 코딩
- NaN은 NaN

In [15]:
data[data['GFR_MDRD'].isnull()].shape

(51, 104)

In [16]:
data['GFR_MDRD'] = data['GFR_MDRD'].apply(lambda x: re.search(r'([0-9]+)\(', str(x)).group(1)
                                         if re.search(r'\(', str(x)) else x)

In [17]:
data[data['GFR_MDRD'].isnull()].shape

(51, 104)

### Organism binarizing
- Organism이 검출되면, 2 그렇지 않으면 1 (='*'), NaN은 NaN

In [18]:
data[data['Organism'].isnull()].shape
#data['Organism'].value_counts()

(18, 104)

In [19]:
data[data['Organism'].str.contains(r'[a-zA-Z0-9]+', na=False)].shape

(43, 104)

In [20]:
data[data['Organism'].str.contains('^\*', na=False)].shape

(1188, 104)

In [21]:
data['Organism'] = data['Organism'].apply(lambda x: 1 if re.search('^\*', str(x))
                                          else (2 if pd.notnull(x) else x))

In [22]:
data[data['Organism']==2].shape

(44, 104)

In [23]:
data[data['Organism']==1].shape

(1188, 104)

#### SPAA binarizing
- 1로 시작하면 1, 2로 시작하면 2, '*'와 NaN은 NaN

In [24]:
data['SPAA'].value_counts()

1                                                                                    1115
2                                                                                      18
*                                                                                       4
2.Splenic a. aneurysms. Large amount of spontaneous portosystemic shunt into LRV.       1
2.Splenic hilum의 아래쪽으로 splenic artery origin의 장경 2 cm aneurysm이 있음(2015.3.31 CT)        1
2. r/o                                                                                  1
2(proximal lig.-회의록에 기입되있음)                                                             1
Name: SPAA, dtype: int64

In [25]:
data['SPAA'] = data['SPAA'].apply(lambda x: 1 if re.search(r'^1', str(x))
                                  else (2 if re.search(r'^2', str(x)) else np.nan))

In [26]:
data['SPAA'].value_counts()

1.0    1115
2.0      22
Name: SPAA, dtype: int64

#### 변수 값이 한 종류만 있는 의미없는 변수 존재하나?

In [27]:
for i, col in enumerate(data.columns) :
    if len(data[col].value_counts()) < 2 :
        print(col, data[col].value_counts())

#### 계급값이 15이하인 것을 categorical variables로 일단 보자

In [28]:
for i, col in enumerate(data.columns) :
    if len(data[col].value_counts()) < 15: 
        print(col)  

Room
Sex
DM_Tx
HTN
CAD
CHF
PCI_CABG
CVA
RCRI
BB
ACEI_ARB
Lasix
Aldactone
CCB
Statin
AntiPLT
AntiCoa
Aspirin
Afib
BBB
HBV
HCV
Alc
Biliary
Toxic
Others
HCC
FHF
Varixbl_LTS
SBP_abd
IntractableAscites
Hydrothorax
HEP
Mental
Loc
preVent
Liver_replace
Pneumonia
Bacteremia
Organism
Sepsis
Pre_Norepinephrine
Pre_Epinephrine
PVT
PVS
SPAA
Biopump
AP
PC
op_Dobu
op_Epi
op_Vaso
op_Terli
op_Iso
op_Ino
PRS
Epi_OX
Levo_mcg
Portography_intraop
Cholangiography_intraop
Splenoportography_intraop
embolization_intraop
Cr_pod
op_bgn_hr
op_mon
op_yr


In [34]:
# Create correlation matrix
corr_matrix = data.corr(method='spearman').abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]
to_drop
upper['Organism'].sort_values(ascending=False)

Bacteremia     0.975631
SBP_abd        0.139186
preHD          0.134222
preVent        0.101835
MELD           0.097935
                 ...   
anes_dur_hr         NaN
op_bgn_hr           NaN
op_ter_hr           NaN
op_mon              NaN
op_yr               NaN
Name: Organism, Length: 86, dtype: float64

#### Organism은 Bacteremia와 거의 동일 따라서 삭제

In [36]:
data = data.drop(columns = ['Organism'])

#### '*' 존재
- 다음 3개의 열에서 Asterisk가 존재하지만 그 뜻을 알 수 없어서 일단 NaN으로 처리

In [37]:
for col in ['Levo_mcg', 'PVT', 'PVS'] :
    data[col] = data[col].replace('*', np.nan)

In [38]:
data = data.replace('없음', np.nan) #Cys_C_GFR, Intraop_Steroid 열에 '없음'이란 값이 존재하지만 무슨 뜻인지 몰라 NaN처리

In [39]:
data.to_csv('troponin_pod1_regression_input.csv', index=False)